In [17]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
%matplotlib inline

In [9]:
boston = load_boston()

In [90]:
boston

{'DESCR': "Boston House Prices dataset\n===========================\n\nNotes\n------\nData Set Characteristics:  \n\n    :Number of Instances: 506 \n\n    :Number of Attributes: 13 numeric/categorical predictive\n    \n    :Median Value (attribute 14) is usually the target\n\n    :Attribute Information (in order):\n        - CRIM     per capita crime rate by town\n        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.\n        - INDUS    proportion of non-retail business acres per town\n        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)\n        - NOX      nitric oxides concentration (parts per 10 million)\n        - RM       average number of rooms per dwelling\n        - AGE      proportion of owner-occupied units built prior to 1940\n        - DIS      weighted distances to five Boston employment centres\n        - RAD      index of accessibility to radial highways\n        - TAX      full-value property-tax rate per

In [15]:
X_data = boston.data
y_target = boston.target

In [46]:
X_data.shape

(506, 13)

In [49]:
y_target.shape

(506,)

In [118]:
train_x, test_x, train_y, test_y = train_test_split(
     boston.data, boston.target, test_size=0.4, random_state=0)

In [129]:
train_max = np.max(train_x, axis=1)
train_max.shape
train_x = train_x/train_max.reshape(13,1)
train_x = train_x.T

In [130]:
train_x[1]

array([ 0.02674131,  0.        ,  0.70583994,  0.        ,  1.        ,
        0.69817768,  1.        ,  0.1170247 ,  0.20833333,  0.56680731,
        0.66818182,  0.4356513 ,  0.75175771])

In [137]:
#test_max = np.max(test_x, axis=0)
#test_max.shape
test_x = test_x.T/test_max.reshape(13,1)
test_x = test_x.T

In [138]:
test_x[0]

array([  9.14405697e-04,   0.00000000e+00,   1.16798846e-01,
         0.00000000e+00,   5.28128588e-01,   7.25845272e-01,
         1.72000000e-01,   4.86877118e-01,   1.66666667e-01,
         6.04781997e-01,   7.97169811e-01,   9.45351474e-01,
         1.93310508e-01])

## Helper functions

In [19]:
# GRADED FUNCTION: sigmoid
def sigmoid(z):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """

    ### START CODE HERE ### (≈ 1 line of code)
    s = 1/(1+np.exp(-z))
    ### END CODE HERE ###
    
    return s

In [20]:
print ("sigmoid([0, 2]) = " + str(sigmoid(np.array([0,2]))))

sigmoid([0, 2]) = [ 0.5         0.88079708]


## Initializing pargameters

In [47]:
# GRADED FUNCTION: initialize_with_zeros
def initialize_with_zeros(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- size of the w vector we want (or number of parameters in this case)
    
    Returns:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias)
    """
    
    ### START CODE HERE ### (≈ 1 line of code)
    w = np.zeros((1,dim))
    b = 0
    ### END CODE HERE ###

    assert(w.shape == (1,dim))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [48]:
dim = 2
w, b = initialize_with_zeros(dim)
print ("w = " + str(w))
print ("b = " + str(b))

w = [[ 0.  0.]]
b = 0


## propagate

In [85]:
# GRADED FUNCTION: propagate
def propagate(w, b, X, Y):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    w -- weights, a numpy array of size (1,n)
    b -- bias, a scalar
    X -- data of size (m, n)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat) of size (m,1)

    Return:
    cost -- negative log-likelihood cost for logistic regression
    dw -- gradient of the loss with respect to w, thus same shape as w
    db -- gradient of the loss with respect to b, thus same shape as b
    
    Tips:
    - Write your code step by step for the propagation. np.log(), np.dot()
    """
    
    m = X.shape[0]
    
    # FORWARD PROPAGATION (FROM X TO COST)
    ### START CODE HERE ### (≈ 2 lines of code)
    A = sigmoid(np.dot(X, w.T) +b)                                # compute activation
    cost = (- np.dot(Y.T, np.log(A)) - np.dot((1-Y).T ,np.log(1-A)))/m                                 # compute cost
    ### END CODE HERE ###

    # BACKWARD PROPAGATION (TO FIND GRAD)
    ### START CODE HERE ### (≈ 2 lines of code)
    dw = np.dot( (A-Y).T, X)/m
    db = np.sum(A-Y)/m 
    ### END CODE HERE ###
    
    #assert(dw.shape == w.shape)
    #assert(db.dtype == float)
    cost = np.squeeze(cost)
    #assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [78]:
w = np.array([[1., 2.]])
b=2.
X= np.array([[1.,3.], [2.,4.], [-1.,-3.2]])
Y= np.array([[1],[0],[1]])
grads, cost = propagate(w, b, X, Y)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

dw = [[ 0.99845601  2.39507239]]
db = 0.00145557813678
cost = 5.801545319394553


## optimize

In [57]:
# GRADED FUNCTION: optimize
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (1, n)
    b -- bias, a scalar
    X -- data of shape (m,n)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat), of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
    Tips:
    You basically need to write down two steps and iterate through them:
        1) Calculate the cost and the gradient for the current parameters. Use propagate().
        2) Update the parameters using gradient descent rule for w and b.
    """
    costs=[]
    for i in range(num_iterations):
        # Cost and gradient calculation 
        grads, cost = propagate(w,b,X,Y)
        # Retrieve derivatives from grads
        dw = grads['dw']
        db = grads['db']
        # update rules
        w = w-learning_rate*dw
        b = b-learning_rate*db
        # Record the costs
        if i%100 ==0:
            costs.append(cost)
        # Print the cost every 100 training iterations
        if print_cost and i % 100 ==0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs    

In [91]:
params, grads, costs = optimize(w, b, X, Y, num_iterations= 1000, learning_rate = 0.009, print_cost = False)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))

w = [[-0.05092733 -0.64613548]]
b = 0.879736991328
dw = [[-0.02172354  0.07079217]]
db = -0.116063355342


## Predict

In [103]:
# GRADED FUNCTION: predict
def predict(w,b,X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size (1,n)
    b -- bias, a scalar
    X -- data of size (m,n)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    m = X.shape[0]
    Y_prediction = np.zeros((m,1))
    #w = w.reshape(1,X.shape[1])
    
    # Compute vector "A" predicting the probabilities
    Y_prediction = sigmoid(np.dot(X, w.T) +b)   
    '''
    for i in range(A.shape[1]):
        if A[0,i]>0.5:
            Y_prediction[0,i]=1
        else:
            Y_prediction[0,i]=0
        pass
    '''
    assert(Y_prediction.shape == (m,1))
    
    return Y_prediction

In [104]:
w = np.array([[0.1124579,0.23106775]])
b = -0.3
X= np.array([[1., 1.2], [-1.1, 2.], [-3.2, 0.1]])
print ("predictions = " + str(predict(w, b, X)))

predictions = [[ 0.52241976]
 [ 0.50960677]
 [ 0.34597965]]


In [105]:
w = np.array([[0.1,0.2,0.2,0.2,0.2,0.2,0.2,0.2,2,0.2,0.2,0.2,0.2]])
b = 10
X= np.array([[   2.37934,    0.     ,   190.58   ,    0.     ,    0.871  ,
          6.13   ,  100.     ,    1.4191 ,    5.     ,  403.     ,
         14.7    ,  172.91   ,   27.8    ]])
print ("predictions = " + str(predict(w, b, X)))

predictions = [[ 1.]]


In [106]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    # initialize parameters with zeros
    w,b = initialize_with_zeros(X_train.shape[1])
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost = False)
    w = parameters["w"]
    b = parameters["b"]
    Y_prediction_test = predict(w,b, X_test)
    Y_prediction_train = predict(w,b, X_train)
     # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [139]:
d = model(train_x, train_y, test_x, test_y, num_iterations = 1000, learning_rate = 0.005, 
          print_cost=True)

/Users/xzy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log


AssertionError: 